| column | description |  
|---|---|
| year | contest year |
| to_country_id | country id of contestant | 
| to_country  | country name of contestant |
| performer | artist |
| song | title of the contestant's song |
| sf_num | participated in semi-final 1, 2 or 0 (from 2004-2008 there was only one semi-final) |
| running_final | order in the broadcast of the contest's final |
| running_sf | order in the broadcast of the contest's semi-final |
| place_final | place in the final |
| points_final | points in the final |
| place_sf | place in the semi-final |
| points_sf | points in the semi-final |
| points_tele_final | televoting points in the contest's final |
| points_jury_final | juryvoting points in the contest's final |
| points_tele_sf | televoting points in the contest's semi-final |
| points_jury_sf | juryvoting points in the contest's semi-final |
| lyrics | lyrics of the song |
| youtube_url | url to video on YouTube |

| column | description |  
|---|---|
| year | contest year |
| round | final, semi-final |
| from_country_id | country id of the country giving points |
| to_country_id | country id of the country receiving points |
| from_country | country name of the country giving points |
| to_country | country name of the country receiving points |
| points | number of points given |

In [76]:
#import the libraries
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np


In [77]:
contestants_final_df = pd.read_csv('./data/contestants.csv')
contestants_final_df = contestants_final_df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)  # replace \\n with \n for newline prints
votes_df=pd.read_csv('./data/votes.csv')
betting_df = pd.read_csv('./data/betting_offices.csv')

In [39]:
contestants_final_df.head()

,year,to_country_id,to_country,performer,song,place_contest,sf_num,running_final,running_sf,place_final,...,place_sf,points_sf,points_tele_final,points_jury_final,points_tele_sf,points_jury_sf,composers,lyricists,lyrics,youtube_url
0,1956,ch,Switzerland,Lys Assia,Refrain,2.0,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Georg Benz Stahl,NaN,"(Refrain d'amour...) Refrain, couleur du ciel...",https://youtube.com/watch?v=IyqIPvOkiRk
1,1956,nl,Netherlands,Jetty Paerl,De Vogels Van Holland,2.0,NaN,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Cor Lemaire,Annie M. G. Schmidt,De vogels van Holland zijn zo muzikaal Ze lere...,https://youtube.com/watch?v=u45UQVGRVPA
2,1956,be,Belgium,Fud Leclerc,Messieurs Les Noyés De La Seine,2.0,NaN,3.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jacques Say;Jean Miret,Robert Montal,Messieurs les noyés de la Seine Ouvrez-moi les...,https://youtube.com/watch?v=U9O3sqlyra0
3,1956,de,Germany,Walter Andreas Schwarz,Im Wartesaal Zum Großen Glück,2.0,NaN,4.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Walter Andreas Schwarz,NaN,"Es gibt einen Hafen, da fährt kaum ein Schiff ...",https://youtube.com/watch?v=BDNARIDnmTc
4,1956,fr,France,Mathé Altéry,Le Temps Perdu,2.0,NaN,5.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,André Lodge,Rachèle Thoreau,"Chante, carillon Le chant du temps perdu Chant...",https://youtube.com/watch?v=dm1L0XyikKI


In [4]:
betting_df.head()

,betting_bm_id,betting_sc_id,betting_name,betting_score,year,performer,song,page_url,contest_round,country_name,country_code
0,5,220,BET365,2.10,2015,Måns Zelmerlöw,Heroes,/eurovision/2015/sweden,final,Sweden,Sweden
1,4,153,UNIBET,2.00,2015,Måns Zelmerlöw,Heroes,/eurovision/2015/sweden,final,Sweden,Sweden
2,18,139,YOUWIN,2.38,2015,Måns Zelmerlöw,Heroes,/eurovision/2015/sweden,final,Sweden,Sweden
3,15,131,BOYLESPORTS,2.25,2015,Måns Zelmerlöw,Heroes,/eurovision/2015/sweden,final,Sweden,Sweden
4,21,124,CORAL,2.10,2015,Måns Zelmerlöw,Heroes,/eurovision/2015/sweden,final,Sweden,Sweden


In [5]:
votes_df.head()

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points
0,1957,final,at,nl,at,nl,6,NaN,NaN
1,1957,final,at,fr,at,fr,0,NaN,NaN
2,1957,final,at,dk,at,dk,0,NaN,NaN
3,1957,final,at,lu,at,lu,3,NaN,NaN
4,1957,final,at,de,at,de,0,NaN,NaN


In [82]:
#Define function for renaming countries based on code
def country_abbrev_mapping(df, col):
    #df =  the Pandas dataframe.
    #col = String. The column with the state name or abbreviation you wish to use
    #List of states
    country_code_list = {'ch':'Switzerland', 'nl':'Netherlands', 'be':'Belgium', 'de':'Germany', 'fr':'France', 'lu':'Luxembourg', 'it':'Italy', 'dk':'Denmark', 
                     'gb':'United Kingdom', 'at':'Austria', 'se':'Sweden','mc':'Monaco', 'no':'Norway', 'yu':'Yugoslavia', 'es':'Spain', 'fi':'Finland', 
                     'pt':'Portugal', 'ie':'Ireland', 'mt':'Malta', 'il':'Israel', 'gr':'Greece', 'tr':'Turkey', 'ma':'Morocco', 'cy':'Cyprus', 'is':'Iceland',
                     'hr':'Croatia', 'ba':'Bosnia & Herzegovina', 'si':'Slovenia', 'pl':'Poland', 'hu':'Hungary', 'ru':'Russia', 'sk':'Slovakia', 'ro':'Romania',
                     'ee': 'Estonia', 'lt':'Lithuania', 'mk':'North Macedonia', 'lv': 'Latvia', 'ua': 'Ukraine', 'cs': 'Czechoslovakia', 'al':'Albania','md':'Moldova', 
                     'am': 'Armenia', 'rs':'Serbia','bg':'Bulgaria', 'by':'Belarus', 'ge':'Georgia','me':'Montenegro', 'sm':'San Marino','au':'Australia', 'cz':'Czech Republic',
                     'az':'Azerbaijan', 'wld':'World', 'ad':'Andorra'}
     
    #If user wants to add a new column
    df[col] = df[col].str.strip().replace(country_code_list)
    return(df)

In [132]:
# mapping round onto data

contestants_final_df['round'] = np.where(contestants_final_df['running_final'].notna(), 'final', 'semifinal')
contestants_final_df.tail(50)

# making song names lower case for join

contestants_final_df['song']=contestants_final_df['song'].str.lower()

#making new column for just country names

contestants_final_df['country'] = contestants_final_df['to_country_id']

# Getting country column for contestants
country_abbrev_mapping(contestants_final_df, 'country')

,year,to_country_id,to_country,performer,song,place_contest,sf_num,running_final,running_sf,place_final,...,points_tele_final,points_jury_final,points_tele_sf,points_jury_sf,composers,lyricists,lyrics,youtube_url,country,round
0,1956,ch,Switzerland,Lys Assia,refrain,2.0,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,Georg Benz Stahl,NaN,"(Refrain d'amour...) Refrain, couleur du ciel...",https://youtube.com/watch?v=IyqIPvOkiRk,Switzerland,final
1,1956,nl,Netherlands,Jetty Paerl,de vogels van holland,2.0,NaN,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,Cor Lemaire,Annie M. G. Schmidt,De vogels van Holland zijn zo muzikaal Ze lere...,https://youtube.com/watch?v=u45UQVGRVPA,Netherlands,final
2,1956,be,Belgium,Fud Leclerc,messieurs les noyés de la seine,2.0,NaN,3.0,NaN,2.0,...,NaN,NaN,NaN,NaN,Jacques Say;Jean Miret,Robert Montal,Messieurs les noyés de la Seine Ouvrez-moi les...,https://youtube.com/watch?v=U9O3sqlyra0,Belgium,final
3,1956,de,Germany,Walter Andreas Schwarz,im wartesaal zum großen glück,2.0,NaN,4.0,NaN,2.0,...,NaN,NaN,NaN,NaN,Walter Andreas Schwarz,NaN,"Es gibt einen Hafen, da fährt kaum ein Schiff ...",https://youtube.com/watch?v=BDNARIDnmTc,Germany,final
4,1956,fr,France,Mathé Altéry,le temps perdu,2.0,NaN,5.0,NaN,2.0,...,NaN,NaN,NaN,NaN,André Lodge,Rachèle Thoreau,"Chante, carillon Le chant du temps perdu Chant...",https://youtube.com/watch?v=dm1L0XyikKI,France,final
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,2023,lv,Latvia,Sudden Lights,aijā,NaN,1.0,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,Andrejs Reinis Zitmanis;Kārlis Matīss Zitmanis...,Andrejs Reinis Zitmanis,Aijā You said some words didn't hear you fall...,https://youtube.com/watch?v=xAjVA3qwqGk,Latvia,semifinal
1730,2023,ie,Ireland,Wild Youth,we are one,NaN,1.0,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,Conor O'Donohoe;Edward Porter;Jörgen Elofsson,NaN,We Are One We take our first breath And then ...,https://youtube.com/watch?v=ak5Fevs424Y,Ireland,semifinal
1731,2023,nl,Netherlands,Mia Nicolai & Dion Cooper,burning daylight,NaN,1.0,NaN,14.0,NaN,...,NaN,NaN,NaN,NaN,Dion Cooper;Duncan Laurence;Jordan Garfield;Lo...,NaN,Burning Daylight I don't find any joy anymore...,https://youtube.com/watch?v=UOf-oKDlO6A,Netherlands,semifinal
1732,2023,az,Azerbaijan,TuralTuranX,tell me more,NaN,1.0,NaN,12.0,NaN,...,NaN,NaN,NaN,NaN,Nihad Aliyev;Tunar Tağiyev;Tural Bağmanov;Tura...,Tural Bağmanov;Turan Bağmanov,Tell Me More Your call has been forwarded to ...,https://youtube.com/watch?v=5dvsr-L3HgY,Azerbaijan,semifinal


In [9]:
contestants_final_df['country'].nunique()

52

In [10]:
contestants_final_df['country'].unique()

array(['Switzerland', 'Netherlands', 'Belgium', 'Germany', 'France',
       'Luxembourg', 'Italy', 'Denmark', 'United Kingdom', 'Austria',
       'Sweden', 'Monaco', 'Norway', 'Yugoslavia', 'Spain', 'Finland',
       'Portugal', 'Ireland', 'Malta', 'Israel', 'Greece', 'Turkey',
       'Morocco', 'Cyprus', 'Iceland', 'Croatia', 'Bosnia & Herzegovina',
       'Slovenia', 'Poland', 'Hungary', 'Russia', 'Slovakia', 'Romania',
       'Estonia', 'Lithuania', 'North Macedonia', 'Latvia', 'Ukraine',
       'Czechoslovakia', 'Albania', 'Andorra', 'Belarus', 'Moldova',
       'Bulgaria', 'Armenia', 'Serbia', 'Georgia', 'Montenegro',
       'Czech Republic', 'Azerbaijan', 'San Marino', 'Australia'],
      dtype=object)

In [11]:
contestants_final_df['to_country_id'].unique()

array(['ch', 'nl', 'be', 'de', 'fr', 'lu', 'it', 'dk', 'gb', 'at', 'se',
       'mc', 'no', 'yu', 'es', 'fi', 'pt', 'ie', 'mt', 'il', 'gr', 'tr',
       'ma', 'cy', 'is', 'hr', 'ba', 'si', 'pl', 'hu', 'ru', 'sk', 'ro',
       'ee', 'lt', 'mk', 'lv', 'ua', 'cs', 'al', 'Estonia', 'Israel',
       'Denmark', 'Finland', 'Portugal', 'Lithuania', 'Latvia', 'Andorra',
       'Belarus', 'Monaco', 'Slovenia', 'Switzerland', 'md', 'Poland',
       'Netherlands', 'Ireland', 'Iceland', 'Bulgaria', 'Austria',
       'Belgium', 'am', 'Albania', 'Cyprus', 'rs', 'bg', 'by', 'ge',
       'Croatia', 'Norway', 'Montenegro', 'Malta', 'Czech Republic', 'az',
       'North Macedonia', 'Moldova', 'Hungary', 'San Marino', 'Serbia',
       'Slovakia', 'Sweden', 'Armenia', 'Turkey', 'Georgia', 'me', 'sm',
       'au', 'cz', 'Bosnia & Herzegovina', 'Greece', 'Romania',
       'Azerbaijan', 'Russia', 'Australia', 'Ukraine'], dtype=object)

In [5]:
country_abbrev_mapping(votes_df, 'from_country')

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points
0,1957,final,at,nl,Austria,nl,6,NaN,NaN
1,1957,final,at,fr,Austria,fr,0,NaN,NaN
2,1957,final,at,dk,Austria,dk,0,NaN,NaN
3,1957,final,at,lu,Austria,lu,3,NaN,NaN
4,1957,final,at,de,Austria,de,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
51349,2023,final,wld,al,World,al,6,6.0,NaN
51350,2023,final,wld,pt,World,pt,0,0.0,NaN
51351,2023,final,wld,rs,World,rs,0,0.0,NaN
51352,2023,final,wld,gb,World,gb,0,0.0,NaN


In [6]:
country_abbrev_mapping(votes_df, 'to_country')

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points
0,1957,final,at,nl,Austria,Netherlands,6,NaN,NaN
1,1957,final,at,fr,Austria,France,0,NaN,NaN
2,1957,final,at,dk,Austria,Denmark,0,NaN,NaN
3,1957,final,at,lu,Austria,Luxembourg,3,NaN,NaN
4,1957,final,at,de,Austria,Germany,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
51349,2023,final,wld,al,World,Albania,6,6.0,NaN
51350,2023,final,wld,pt,World,Portugal,0,0.0,NaN
51351,2023,final,wld,rs,World,Serbia,0,0.0,NaN
51352,2023,final,wld,gb,World,United Kingdom,0,0.0,NaN


In [14]:
votes_df['from_country'].nunique()

53

In [15]:
votes_df['to_country'].nunique()

52

In [7]:
votes_df['relationship']=(votes_df['from_country']+"-"+votes_df['to_country'])
votes_df.head()

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points,relationship
0,1957,final,at,nl,Austria,Netherlands,6,NaN,NaN,Austria-Netherlands
1,1957,final,at,fr,Austria,France,0,NaN,NaN,Austria-France
2,1957,final,at,dk,Austria,Denmark,0,NaN,NaN,Austria-Denmark
3,1957,final,at,lu,Austria,Luxembourg,3,NaN,NaN,Austria-Luxembourg
4,1957,final,at,de,Austria,Germany,0,NaN,NaN,Austria-Germany


In [30]:
# defining data types for the votes_df
dtype_dict_votes = {
    'year':types.Integer(),
    'round': types.String(),
    'from_country_id': types.String(),
    'to_country_id': types.String(),
    'from_country': types.String(),
    'to_country': types.String(),
    'total_points': types.Integer(),
    'tele_points': types.Integer(),
    'jury_points': types.Integer(),
    'relationship': types.String()
             }

In [24]:
contestants_final_df.head(1)

,year,to_country_id,to_country,performer,song,place_contest,sf_num,running_final,running_sf,place_final,...,points_sf,points_tele_final,points_jury_final,points_tele_sf,points_jury_sf,composers,lyricists,lyrics,youtube_url,country
0,1956,ch,Switzerland,Lys Assia,Refrain,2.0,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,Georg Benz Stahl,NaN,"(Refrain d'amour...)\n\nRefrain, couleur du ci...",https://youtube.com/watch?v=IyqIPvOkiRk,Switzerland


In [ ]:
# defining data types for the contestants_final_df

dtype_dict_votes = {
    'year':types.DateTime,



In [32]:
# Pushing these tables to the AWS

from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string


# Let's load values from the .env file
from dotenv import dotenv_values

config = dotenv_values()

# define variables for the login
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']


url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

engine = create_engine(url, echo=False)

In [41]:
#update pg_schema to believe_in_all_butts

votes_df.to_sql(name = 'votes', 
                       con = engine, 
                       schema = pg_schema, # pandas is allowing to specify, in which schema the table shall be created
                       if_exists='replace', 
                       dtype=dtype_dict_votes,
                       index=False
                      )

#update pg_schema to believe_in_all_butts

contestants_final_df.to_sql(name = 'contestants_final', 
                       con = engine, 
                       schema = pg_schema, # pandas is allowing to specify, in which schema the table shall be created
                       if_exists='replace', 
                       index=False
                      )

734

In [20]:
#update pg_schema to believe_in_all_butts

betting_df.to_sql(name = 'betting', 
                       con = engine, 
                       schema = pg_schema, # pandas is allowing to specify, in which schema the table shall be created
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

453

In [55]:
contestants_final_df['song']

np.int64(56)

In [148]:
# adding LGBTQ id:

lgbtq_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_LGBTQ_participants_in_the_Eurovision_Song_Contest')
lgbtq_df = lgbtq_df[0]
lgbtq_df['Points']=lgbtq_df['Points'].replace('—[a]', 'not available')
lgbtq_df['Place']=lgbtq_df['Place'].replace('—[a]', 'not available')
# lgbtq_df['sexual_orientation'] = lgbtq_df['Sexual orientation or gender identity']
# lgbtq_df['gender_identity'] = lgbtq_df['Sexual orientation or gender identity']
# for entry in lgbtq_df['Sexual orientation or gender identity']:
#     if entry == 'Bisexual[w][50] and non-binary gender[b][51]':
#         lgbtq_df['sexual_orientation']='Bisexual'
#         lgbtq_df['gender_identity']='Non-binary'
#     elif entry == 'Non-binary gender[w][76] and pansexual[77]':
#         lgbtq_df['sexual_orientation']='Pansexual'
#         lgbtq_df['gender_identity']='Non-binary'
#     elif entry == 'Gay[78] and Non-binary gender[79]':
#         lgbtq_df['sexual_orientation']='Gay'
#         lgbtq_df['gender_identity']='Non-binary'
#     elif entry.startswith('Sexually free'):
#         lgbtq_df['sexual_orientation']='Sexually free'
#     elif entry.startswith('Lesbian'):
#         lgbtq_df['sexual_orientation']='Lesbian'
#     elif entry.startswith('Gay'):
#         lgbtq_df['sexual_orientation']='Gay'
#     elif entry.startswith('Bisexual'):
#         lgbtq_df['sexual_orientation']='Bisexual'
#     elif entry.startswith('Trans woman'):
#         lgbtq_df['gender_identity']='Trans woman'
#     elif entry.startswith('Non-binary gender'):
#        lgbtq_df['gender_identity']='Non-binary'
#     elif entry.startswith('Intersex'):
#         lgbtq_df['gender_identity']='Intersex'
#     elif entry.startswith('Pansexual'):
#         lgbtq_df['sexual_orientation']='Pansexual'
#     elif entry.startswith('Queer'):
#         lgbtq_df['sexual_orientation']='Queer'
lgbtq_df.columns = lgbtq_df.columns.str.lower()
lgbtq_df['lgbtqia+']=lgbtq_df['sexual orientation or gender identity']
lgbtq_df['lgbtqia+']=1
# lgbtq_df=lgbtq_df.drop(['sexual orientation or gender identity','points','place', 'country', 'year', 'artist'], axis='columns')
lgbtq_df['song']=lgbtq_df['song'].str.replace('"', ' ').str.strip()
lgbtq_df['song']=lgbtq_df['song'].str.lower()

In [150]:
lgbtq_df[null_filter].to_csv('missing_names_with_info.csv')

/var/folders/bb/x10xqm6x59b9vj37y_w2nzzm0000gn/T/ipykernel_25487/3996538457.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lgbtq_df[null_filter].to_csv('missing_names_with_info.csv')


In [131]:
lgbtq_df.head()

,song,lgbtqia+
0,il est là,1
1,dors mon amour,1
2,hou toch van mij,1
3,"september, gouden roos",1
4,nous les amoureux,1


In [135]:
contestants_final_enhanced_df = pd.merge(right=lgbtq_df, left=contestants_final_df, how='left', on='song')

In [139]:
test_songs_df = pd.merge(right=lgbtq_df, left=contestants_final_df, how='right', on='song')

In [143]:
null_filter = test_songs_df['performer'].isnull()
test_songs_df[null_filter]
test_songs_df[null_filter].to_csv('missing_names.csv')

In [104]:
contestants_final_enhanced_df['lgbtqia+'] = contestants_final_enhanced_df['lgbtqia+'].fillna(value=0')

In [136]:
contestants_final_enhanced_df['lgbtqia+'].sum()

np.float64(90.0)

In [115]:
contestants_final_enhanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734 entries, 0 to 1733
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               1734 non-null   int64  
 1   to_country_id      1734 non-null   object 
 2   to_country         1734 non-null   object 
 3   performer          1734 non-null   object 
 4   song               1731 non-null   object 
 5   place_contest      1678 non-null   float64
 6   sf_num             640 non-null    float64
 7   running_final      1398 non-null   float64
 8   running_sf         605 non-null    float64
 9   place_final        1397 non-null   float64
 10  points_final       1385 non-null   float64
 11  place_sf           605 non-null    float64
 12  points_sf          605 non-null    float64
 13  points_tele_final  181 non-null    float64
 14  points_jury_final  181 non-null    float64
 15  points_tele_sf     212 non-null    float64
 16  points_jury_sf     212 n